## Import Lib

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## FILES

In [ ]:
train=pd.read_csv("/content/Train_aBjfeNk.csv")

In [ ]:
test=pd.read_csv("/content/Test_LqhgPWU.csv")

## EXPLORE

In [ ]:
train.tail(3)

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
8065,465406,Female,No,33,Yes,Healthcare,1.0,Low,1.0,Cat_6,D
8066,467299,Female,No,27,Yes,Healthcare,1.0,Low,4.0,Cat_6,B
8067,461879,Male,Yes,37,Yes,Executive,0.0,Average,3.0,Cat_4,B


In [ ]:
test.head(3)

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
0,458989,Female,Yes,36,Yes,Engineer,0.0,Low,1.0,Cat_6
1,458994,Male,Yes,37,Yes,Healthcare,8.0,Average,4.0,Cat_6
2,458996,Female,Yes,69,No,NaN,0.0,Low,1.0,Cat_6


## CONCAT Train-Test

In [ ]:
data=pd.concat([train,test],ignore_index=True)

In [ ]:
len(train),len(test)

(8068, 2627)

In [ ]:
len(np.unique(data["ID"]))

8363

## Common ids to train-test dataset

In [ ]:

train_test_common = pd.merge(train,test, how='inner', on=['ID']).sort_values("ID")

In [ ]:
yy=train_test_common[["ID","Segmentation"]]

## Non-common ids to train-test dataset

In [ ]:
common = train.merge(test, on=["ID"])
non_common_train = train[~train.ID.isin(common.ID)]
non_common_test=test[~test.ID.isin(common.ID)]

In [ ]:
len(non_common_train)

5736

## Target Variable

In [ ]:
y=train.pop("Segmentation")

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
target=le.fit_transform(y)
from sklearn.model_selection import cross_val_score

## Concat non-common data

In [ ]:
non_common_data=pd.concat([train1,non_common_test])

In [ ]:
non_common_data.isnull().sum()

ID                     0
Gender                 0
Ever_Married           0
Age                    0
Graduated              0
Profession             0
Work_Experience        0
Spending_Score         0
Family_Size            0
Var_1                  0
GEnder_enco            0
everM_enco             0
Grad_enco              0
Prof_enco              0
WE_enco                0
Spending_Score_enco    0
Family_Size_enco       0
Var_1_enco             0
dtype: int64

## Missing Values

In [ ]:
non_common_data.Ever_Married.value_counts()
non_common_data.Ever_Married=non_common_data.Ever_Married.fillna("Yes")

In [ ]:
non_common_data.Graduated.value_counts()
non_common_data.Graduated=non_common_data.Graduated.fillna("Yes")

In [ ]:
non_common_data.Profession.value_counts()
non_common_data.Profession=non_common_data.Profession.fillna("Artist")

In [ ]:
non_common_data.Work_Experience.value_counts()
non_common_data.Work_Experience=non_common_data.Work_Experience.fillna(1.0)

In [ ]:
non_common_data.Family_Size=non_common_data.Family_Size.fillna(2.0)

In [ ]:
non_common_data.Var_1.value_counts()
non_common_data.Var_1=non_common_data.Var_1.fillna("Cat_6")

In [ ]:
non_common_data.Age=np.log1p(non_common_data.Age)

In [ ]:
train["segment"]=target

## Mean Encoding

In [ ]:
"""
Mean_encoded_subject = train.groupby(['Gender'])['segment'].mean().to_dict()
non_common_data['GEnder_enco'] =  non_common_data['Gender'].map(Mean_encoded_subject)

Mean_encoded_subject = train.groupby(['Ever_Married'])['segment'].mean().to_dict()
non_common_data['everM_enco'] =  non_common_data['Ever_Married'].map(Mean_encoded_subject)

Mean_encoded_subject = train.groupby(['Graduated'])['segment'].mean().to_dict()
non_common_data['Grad_enco'] =  non_common_data['Graduated'].map(Mean_encoded_subject)

Mean_encoded_subject = train.groupby(['Profession'])['segment'].mean().to_dict()
non_common_data['Prof_enco'] =  non_common_data['Profession'].map(Mean_encoded_subject)

Mean_encoded_subject = train.groupby(['Work_Experience'])['segment'].mean().to_dict()
non_common_data['WE_enco'] =  non_common_data['Work_Experience'].map(Mean_encoded_subject)

Mean_encoded_subject = train.groupby(['Spending_Score'])['segment'].mean().to_dict()
non_common_data['Spending_Score_enco'] =  non_common_data['Spending_Score'].map(Mean_encoded_subject)

Mean_encoded_subject = train.groupby(['Family_Size'])['segment'].mean().to_dict()
non_common_data['Family_Size_enco'] =  non_common_data['Family_Size'].map(Mean_encoded_subject)

Mean_encoded_subject = train.groupby(['Var_1'])['segment'].mean().to_dict()
non_common_data['Var_1_enco'] =  non_common_data['Var_1'].map(Mean_encoded_subject)   """

## train-test-split

In [ ]:
non_common_df=pd.get_dummies(non_common_data)

In [ ]:
trainxx=non_common_df[:8068]
testxx=non_common_df[8068:]

# MODEL-lightgbm

In [ ]:
from lightgbm import LGBMClassifier
lgb=LGBMClassifier()
scores = cross_val_score(lgb, trainxx,target, cv = 5, scoring = "accuracy")
(scores.mean())

0.540532737800292

# MODEL-xgboost

In [ ]:
from xgboost import XGBClassifier
xgb=XGBClassifier()
scores = cross_val_score(xgb, trainxx,target, cv = 5, scoring = "accuracy")
(scores.mean())

0.5379287403846228

# MODEL-catboost

In [ ]:
pip install catboost

     |████████████████████████████████| 64.8MB 56kB/s 


In [ ]:
from catboost import CatBoostClassifier
cat=CatBoostClassifier()
scores = cross_val_score(cat, trainxx,target, cv = 5, scoring = "accuracy")
(scores.mean())

Streaming output truncated to the last 5000 lines.
5:	learn: 1.2089604	total: 63.5ms	remaining: 10.5s
6:	learn: 1.1927971	total: 73.8ms	remaining: 10.5s
7:	learn: 1.1797014	total: 83.4ms	remaining: 10.3s
8:	learn: 1.1672619	total: 92.9ms	remaining: 10.2s
9:	learn: 1.1574928	total: 103ms	remaining: 10.2s
10:	learn: 1.1474842	total: 113ms	remaining: 10.1s
11:	learn: 1.1404396	total: 123ms	remaining: 10.1s
12:	learn: 1.1330680	total: 132ms	remaining: 10s
13:	learn: 1.1237922	total: 142ms	remaining: 9.99s
14:	learn: 1.1176453	total: 151ms	remaining: 9.91s
15:	learn: 1.1116614	total: 160ms	remaining: 9.86s
16:	learn: 1.1054627	total: 170ms	remaining: 9.82s
17:	learn: 1.1005022	total: 186ms	remaining: 10.2s
18:	learn: 1.0946536	total: 202ms	remaining: 10.4s
19:	learn: 1.0896955	total: 212ms	remaining: 10.4s
20:	learn: 1.0857557	total: 222ms	remaining: 10.3s
21:	learn: 1.0817612	total: 231ms	remaining: 10.3s
22:	learn: 1.0793009	total: 240ms	remaining: 10.2s
23:	learn: 1.0760127	total: 250ms	

0.5261535187690474

# MODEL-voting

In [ ]:
from sklearn.ensemble import VotingClassifier
eclf = VotingClassifier(estimators=[ ('lgb', lgb), ("xgb",xgb)], voting='soft')
scores = cross_val_score(eclf, trainxx,target, cv = 5, scoring = "accuracy")
(scores.mean())

0.5422670203604388

## Predict

In [ ]:
eclf.fit(trainxx,target)
pred=eclf.predict(testxx)
predX=le.inverse_transform(pred)

In [ ]:
non_common_test["Segmentation"]=predX

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
yy=train_test_common[["ID","Segmentation"]]

In [ ]:
zz=non_common_test[["ID","Segmentation"]]

In [ ]:
all=pd.concat([yy,zz])

In [ ]:
seg=all.set_index("ID").to_dict()

In [ ]:
sub=pd.read_csv("/content/sample_submission_wyi0h0z.csv")

In [ ]:
sub["Segmentation"]=sub["ID"].map(seg["Segmentation"])

## Submission

In [ ]:
sub.isnull().sum()

ID              0
Segmentation    0
dtype: int64

In [ ]:
sub.to_csv("tt24.csv",index=False)